In [86]:
import sys
import os
from dotenv import load_dotenv

#1. load environment variables and data

# load environment variables
load_dotenv()

#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils.dataset import get_data 
df = get_data()
df.head(10)

Loading data from wines: 8000it [00:00, 17023.44it/s]


,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6
5,Merlot,7.3,0.34,0.30,1.3,22.403749,1044.95,289.523749,267.12,0.057,25.0,173.0,0.99480,3.26,0.51,9.1,6
6,Merlot,7.6,0.21,0.49,2.5,23.875866,888.61,133.545866,109.67,0.047,20.0,130.0,0.99178,3.15,0.48,11.1,5
7,Chardonnay,6.0,0.25,0.40,5.7,23.309699,1381.79,266.529699,243.22,0.052,56.0,152.0,0.99398,3.16,0.88,10.5,6
8,Cabernet Sauvignon,6.7,0.18,0.19,4.7,49.165745,1456.41,269.915745,220.75,0.046,57.0,161.0,0.99460,3.32,0.66,10.5,6
9,Gamay,7.7,0.28,0.39,8.9,54.450579,929.44,377.690579,323.24,0.036,8.0,117.0,0.99350,3.06,0.38,12.0,2


In [88]:
#import pipeline from scikit
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[])

categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns

In [89]:
from sklearn.preprocessing import FunctionTransformer

def outlier_detection(df):
    ### YOUR CODE HERE ###
    return df

outlier_detection = FunctionTransformer(outlier_detection)

pipeline.steps.append(('outlier_detection', outlier_detection))

In [90]:
# simpleimputer for numerical features
from sklearn.impute import SimpleImputer

categorical_imputer = SimpleImputer(strategy="most_frequent")

numerical_imputer = SimpleImputer(strategy="mean")


In [91]:
#impiute missing values
df_missing[numerical_features] = numerical_imputer.fit_transform(df_missing[numerical_features])

_________

Remember to scale the data to the range between 0 and 1 before using PCA!

In [92]:
#MinMaxScaler to normalize numerical features
from sklearn.preprocessing import MinMaxScaler
scaler_minmax = MinMaxScaler()

Scales numerical features to range between 0 and 1.

In [ ]:
#ALTERNATIVE StandardScaler to normalize numerical features
from sklearn.preprocessing import StandardScaler
scaler_std = StandardScaler()

The standard score of a sample `x` is calculated as: 

`z = (x - u) / s `

where `u` is the mean of the training samples or zero if `with_mean=False`, and `s` is the standard deviation of the training samples or one if `with_std=False`.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler


In [93]:
#dimensionality reduction with PCA
from sklearn.decomposition import PCA
#we want the explained variance to be 95%. 
#with choosing n_components=0.95,
#PCA will return the minimum number of principal components 
#at which 95% of the variance is retained.
pca = PCA(n_components=0.95)

PCA wirklich notwendig? 

________

In [94]:
from sklearn.preprocessing import OneHotEncoder

#pipeline for categorical features
categorical_pipeline = Pipeline(steps=[])

categorical_pipeline.steps.append(('imputer', categorical_imputer))
categorical_pipeline.steps.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)))

#pipeline for numerical features
numeric_pipeline = Pipeline(steps=[])
numeric_pipeline.steps.append(('imputer', numerical_imputer))
numeric_pipeline.steps.append(('scaler', scaler_minmax))
numeric_pipeline.steps.append(('pca', pca))

# let pipeline know which pipeline to use for which features
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]).set_output(transform="pandas")

pipeline.steps.append(('preprocessor', preprocessor))

In [95]:
pipeline

Pipeline(steps=[('outlier_detection',
                 FunctionTransformer(func=<function outlier_detection at 0x000001C60EE5F380>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.95))]),
                                                  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'magnesium', 'flavanoids', 'minerals', 'calcium', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol', 'quality'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['wine type'], dtype='object'))]))])

In [96]:
end_data = pipeline.fit_transform(df)

In [97]:
end_data

,num__pca0,num__pca1,num__pca2,num__pca3,num__pca4,num__pca5,num__pca6,num__pca7,num__pca8,num__pca9,num__pca10,cat__wine type_Cabernet Sauvignon,cat__wine type_Chardonnay,cat__wine type_Gamay,cat__wine type_Merlot,cat__wine type_Pinot noir
0,-0.219450,-0.165822,-0.328214,0.060889,-0.028767,-0.054703,-0.195028,-0.127485,-0.010221,-0.003676,0.171867,0.0,0.0,0.0,0.0,1.0
1,0.446373,0.117396,0.042497,-0.172085,0.126128,0.064460,-0.005865,-0.179830,-0.000913,0.036268,0.077103,0.0,0.0,0.0,1.0,0.0
2,-0.347676,0.349928,-0.113221,-0.067233,-0.087502,-0.079108,-0.076625,0.093881,-0.004010,0.065912,-0.042580,0.0,1.0,0.0,0.0,0.0
3,0.380518,-0.383049,0.042327,-0.075316,-0.075302,0.135269,0.112027,0.079060,0.002266,0.075794,-0.182575,0.0,0.0,0.0,1.0,0.0
4,0.515459,-0.128522,-0.337194,-0.022052,-0.083138,-0.150299,0.128821,0.048551,0.004131,-0.078350,-0.101896,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-0.440294,-0.234979,-0.040194,-0.170424,0.111573,0.043694,0.010470,-0.106497,0.010015,0.172414,0.077283,1.0,0.0,0.0,0.0,0.0
7996,-0.449768,0.285344,-0.099251,-0.212353,-0.046189,-0.064095,-0.001597,0.045655,0.018865,0.121827,-0.090604,1.0,0.0,0.0,0.0,0.0
7997,-0.040554,0.278476,0.157926,-0.165646,-0.055998,-0.049885,-0.034065,0.014476,-0.000920,-0.026939,-0.066973,0.0,0.0,0.0,1.0,0.0
7998,-0.357420,0.404444,-0.063735,-0.065792,0.185052,-0.145802,0.108557,0.141782,-0.008563,-0.026582,0.047042,0.0,1.0,0.0,0.0,0.0
